In [110]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [111]:
#Cleaned and imported Amazon data
csv_file = "Resources/Amazon_data.csv"
amazon_df = pd.read_csv(csv_file)
amazon_df.head()

,amazon_title,amazon_author,amazon_rating,amazon_reviews count,amazon_isbn-10,amazon_price
0,Tell Me your Dreams,by Sidney Sheldon,4.4,160.0,8172234902,209
1,The Boy in the Striped Pyjamas (Definitions),by John Boyne,4.6,134.0,1862305277,350
2,Romancing the Balance Sheet: For Anyone Who Ow...,by Anil Lamba,4.5,156.0,9350294311,477
3,Mossad,by Michael Bar-Zohar - Import,4.6,637.0,8184958455,340
4,My Story,by Kamala Das,4.5,42.0,8172238975,178


In [112]:
#Renamed amazon_isbn-10 to isbn. This is where we will merge with goodread data

amazon_df = amazon_df.rename(columns = {"amazon_isbn-10":"isbn"})
amazon_df = amazon_df.rename(columns = {"amazon_author":"authors"})
amazon_df = amazon_df.rename(columns = {"amazon_title":"title"})
amazon_df.head()

,title,authors,amazon_rating,amazon_reviews count,isbn,amazon_price
0,Tell Me your Dreams,by Sidney Sheldon,4.4,160.0,8172234902,209
1,The Boy in the Striped Pyjamas (Definitions),by John Boyne,4.6,134.0,1862305277,350
2,Romancing the Balance Sheet: For Anyone Who Ow...,by Anil Lamba,4.5,156.0,9350294311,477
3,Mossad,by Michael Bar-Zohar - Import,4.6,637.0,8184958455,340
4,My Story,by Kamala Das,4.5,42.0,8172238975,178


In [113]:
#Included only thr rows from Amazon data that we will be using to merge
amazon_df = amazon_df[["title", "authors", "amazon_rating"]]
#Have to convert the isbn to a string to merge later on.
#amazon_df["isbn"] = amazon_df["isbn"].astype(str)
amazon_df["title"] = amazon_df["title"].astype(str)
amazon_df["authors"] = amazon_df["authors"].astype(str)
#amazon_df["amazon_rating"] = amazon_df["amazon_rating"].astype(str)
#amazon_df.dropna(subset=['amazon_rating'])
amazon_df["authors"] = amazon_df["authors"].str.replace("by ","")
amazon_df["amazon_rating"] = amazon_df["amazon_rating"].str[0:3]
print (amazon_df.dtypes)
amazon_df.sort_values(by=['amazon_rating'], ascending=False)

title            object
authors          object
amazon_rating    object
dtype: object


,title,authors,amazon_rating
1020,Jeevan Ke Prishth: Buddh Kee Deshanaa,Osho,5.0
560,"Textbook of Anatomy: Head, Neck and Brain - Vol.3",Vishram Singh,5.0
1280,"Textbook of Anatomy: Head, Neck and Brain - Vol.3",Vishram Singh,5.0
1267,Solved Papers for Manipal Engineering 2018,Arihant Experts,5.0
1266,Communication and Educational Technology in Nu...,Sharma Suresh,5.0
1253,50 Successful Harvard Application Essays: What...,Harvard Crimson,5.0
608,A Dog's Way Home: The Heartwarming Story of th...,W. Bruce Cameron,5.0
1244,Free Hit: The Story of Women's Cricket in India,Suprita Das,5.0
1236,Ronaldo - 2018 Updated Edition: The Obsession ...,Luca Caioli,5.0
131,Current Affairs Yearly 2019,Not Available,5.0


### Create new data with select columns

In [114]:
#Cleaned and imported .csv from goodread and created a dataframe

csv_file = "Resources/books.csv"
goodreads_df = pd.read_csv(csv_file)
goodreads_df.head(50)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964
5,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,38872,154
6,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,0976540606,9780976540601,en-US,152,18,1
7,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,27410,820
8,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3602,258
9,13,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,0345453743,9780345453747,eng,815,240189,3954


In [115]:
#Included only thr rows from goodreads data that we will be using to merge

goodreads_df = goodreads_df[["title", 
    "authors", 
    "average_rating"
]]
#goodreads_df.sort_values(by = "average_rating", ascending = False)

print (goodreads_df.dtypes)

goodreads_df.sort_values(by = ['average_rating'], ascending = False)

#goodreads_df.sort_values(by=['title'], ascending=True)

title              object
authors            object
average_rating    float64
dtype: object


,title,authors,average_rating
6079,The Complete Theory Fun Factory: Music Theory ...,Ian Martin-Katie Elliott,5.00
4940,Zone of the Enders: The 2nd Runner Official St...,Tim Bogenn,5.00
963,Literature Circle Guide: Bridge to Terabithia:...,Tara MacCarthy,5.00
7509,The New Big Book of America,Todd Davis-Marc Frey,5.00
11072,Bubbles in the Middle (The Powerpuff Girls Plu...,Aaron Rosenberg-Christopher Cook,5.00
13055,Ecuador Nature Guide: Southwest Forests: Sozo...,Chris Jiggins-Pablo Andrade-Eduardo Cueva,5.00
879,Willem de Kooning: Late Paintings,Julie Sylvester-David Sylvester,5.00
7437,Taxation of Mineral Rents,Ross Garnaut,5.00
11547,The Irish Anatomist: A Study of Flann O'Brien,Keith Donohue,5.00
8149,Delwau Duon: Peintiadau Nicholas Evans = Symph...,Nicholas Evans-Rhonda Evans,5.00


In [116]:
list(goodreads_df)

['title', 'authors', 'average_rating']

In [117]:
amazon_df.describe()

,title,authors,amazon_rating
count,1382,1382,1378
unique,703,474,25
top,The Power of Your Subconscious Mind: Unlock Yo...,Arihant Experts,4.4
freq,12,41,229


In [132]:
merged_df = pd.merge(goodreads_df,amazon_df, how = "inner", on =["title", "authors"])
merged_df


,title,authors,average_rating,amazon_rating
0,Think and Grow Rich,Napoleon Hill,4.18,4.6
1,Think and Grow Rich,Napoleon Hill,4.18,4.6
2,Think and Grow Rich,Napoleon Hill,4.18,4.6
3,Think and Grow Rich,Napoleon Hill,4.18,4.6
4,Think and Grow Rich,Napoleon Hill,4.18,4.6
5,Think and Grow Rich,Napoleon Hill,4.18,4.6
6,Little Women,Louisa May Alcott,4.06,4.3
7,Little Women,Louisa May Alcott,4.06,4.3
8,The Fifth Mountain,Paulo Coelho,3.62,4.4
9,The Great Gatsby,F. Scott Fitzgerald,3.91,4.1


In [134]:
merged_df.describe()

,average_rating
count,60.000000
mean,4.043000
std,0.145722
min,3.620000
25%,3.910000
50%,4.060000
75%,4.180000
max,4.230000


In [124]:
merged_df1 = merged_df.groupby('title')
merged_df1.mean()

,average_rating
title,
Dracula,3.99
How to Win Friends and Influence People,4.18
It,4.23
Little Women,4.06
Nothing Lasts Forever,3.83
The Art of War,3.97
The Fifth Mountain,3.62
The Great Gatsby,3.91
The Origin of Species,3.99


In [120]:
merged_df1 = merged_df[["title", 
    "authors", 
    "average_rating",
    "amazon_rating"
]]

merged_df1.sort_values(by=['average_rating'], 
    ascending=True)

merged_df1.drop_duplicates(subset ="title", 
    keep = False, inplace = True) 

merged_df1.head(100)

,title,authors,average_rating,amazon_rating
8,The Fifth Mountain,Paulo Coelho,3.62,4.4
26,The Power of Now: A Guide to Spiritual Enlight...,Eckhart Tolle,4.13,4.5
27,It,Stephen King,4.23,4.4
